In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from mnist import read_data_sets
import scipy.misc
import os

input_data = read_data_sets('MNIST_data', one_hot=True)

filepath:MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-images-idx3-ubyte.gz
filepath:MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
filepath:MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
filepath:MNIST_data\t10k-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,[None,10])

In [3]:
img = tf.reshape(x,[-1,28,28,1])

以下代码用于构建抽象的权重w，偏置b生成函数，其中：
    权重参数w用truncated_normal方法生成，这是一个产生正态分布序列的函数
    偏置参数b用常数0.1进行初始化

In [4]:
def weight(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

以下代码用于构建抽象的卷积层和池化层操作，其中：
    
    conv函数内部调用tensorflow提供的tf.nn.conv2d函数进行卷积计算，具体参数解释：
        x为输入tensor
        w为卷积核(filter/kernel)
        strides参数用于指定卷积核滑动时的步长，如[1,1,1,1]表示卷积核每次在图片上移动一个像素的距离
        padding参数用于指定当滑动到最后如果卷积核超出图像范围时的处理方法，same为使用0像素填充超出的部分
    
    pool函数内部调用tf.nn.max_pool函数进行池化操作，具体参数如下：
        x为卷积后产生的“图片”
        ksize表示池化操作的范围，如[1,2,2,1]则表示在二维2x2的小方块上进行池化
        strides表示池化操作的滑动步长
        padding参数意义同tf.nn.conv2d函数

In [5]:
def conv(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding="SAME")

def pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

以下代码用于构建卷积网络流程，具体架构及参数如下：
    
    网络架构：
    卷积层1 - relu激活 - 最大池化 - 卷积层2 - relu激活 - 最大池化
    
    参数为：
    卷积层1：卷积核大小：5*5 由于是灰度图片，通道数为1，人工设定32个卷积核
    卷积层2：卷积核大小：5*5 由于第一层卷积产生32个feature_map，通道数为32，人工设定64个卷积核
       

In [6]:
conv1_w = weight([5,5,1,32])
conv1_b = bias([32])
conv1_relu = tf.nn.relu(conv(img,conv1_w) + conv1_b)
pool1_pool = pool(conv1_relu) 

conv2_w = weight([5,5,32,64])
conv2_b = bias([64])
conv2_relu = tf.nn.relu(conv(pool1_pool,conv2_w) + conv2_b)
pool2_pool = pool(conv2_relu)

以下代码用于构建全连接层：
    
    首先将两层卷积之后的feature_map展开成一维，由于经历两次最大池化，因此大小变成原来的1/4，即28/4=7
    人工定义神经元个数为1024个

In [7]:
fc1_w = weight([7*7*64,1024])
fc1_b = bias([1024])
fc1_flat = tf.reshape(pool2_pool,[-1,7*7*64])
fc1_relu = tf.nn.relu(tf.matmul(fc1_flat,fc1_w) + fc1_b)
keep_prob = tf.placeholder(tf.float32)
fc1_drop = tf.nn.dropout(fc1_relu,keep_prob)

fc2_w = weight([1024,10])
fc2_b = bias([10])
y_fc = tf.matmul(fc1_drop,fc2_w) + fc2_b


以下代码用于构建训练目标以及优化函数：
    
    tf.nn.softmax_cross_entropy_with_logits：计算y_和y_fc的交叉熵
    tf.train.AdamOptimizer：指明使用Adam优化器，目标是最小化cross_entropy

In [8]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_fc))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [9]:
correct_prediction = tf.equal(tf.argmax(y_,1),tf.argmax(y_fc,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(10000):
    batch = input_data.train.next_batch(64)
    if i %100 == 0:
        train_acc = accuracy.eval(feed_dict={
            x:batch[0],y_:batch[1],keep_prob:1.0})
        print("step %d,training accuracy %g" % (i,train_acc))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})
print("test accuracy %g" % accuracy.eval(feed_dict={x:input_data.test.images,y_:input_data.test.labels,keep_prob:1.0}))

step 0,training accuracy 0.109375
step 100,training accuracy 0.921875
step 200,training accuracy 0.8125
step 300,training accuracy 0.859375
step 400,training accuracy 0.828125
step 500,training accuracy 0.875
step 600,training accuracy 0.96875
step 700,training accuracy 0.96875
step 800,training accuracy 0.921875
step 900,training accuracy 0.953125
step 1000,training accuracy 0.96875
step 1100,training accuracy 0.984375
step 1200,training accuracy 0.96875
step 1300,training accuracy 0.984375
step 1400,training accuracy 0.96875
step 1500,training accuracy 0.984375
step 1600,training accuracy 0.984375
step 1700,training accuracy 0.984375
step 1800,training accuracy 0.96875
step 1900,training accuracy 0.984375
step 2000,training accuracy 1
step 2100,training accuracy 0.96875
step 2200,training accuracy 0.984375
step 2300,training accuracy 0.96875
step 2400,training accuracy 0.984375
step 2500,training accuracy 0.984375
step 2600,training accuracy 0.984375
step 2700,training accuracy 1
ste